## Taller Regularización y SVM


Para esta tarea consideramos un conjunto de observaciones sobre una serie de variedades de vino tinto y blanco que implican sus propiedades químicas y su clasificación por parte de los catadores. La industria del vino ha experimentado un gran crecimiento en los últimos tiempos debido al aumento del consumo social. El precio del vino depende de un concepto bastante abstracto de apreciación del vino por parte de los catadores, cuya opinión puede tener un alto grado de variabilidad. El precio del vino depende en cierta medida de este factor tan volátil. Otro factor clave en la certificación y evaluación de la calidad del vino son las pruebas fisicoquímicas, que se realizan en laboratorio y tienen en cuenta factores como la acidez, el nivel de pH, la presencia de azúcar y otras propiedades químicas. Para el mercado del vino, sería interesante que la calidad humana de la cata pudiera relacionarse con las propiedades químicas del vino para que el proceso de certificación y evaluación de la calidad estuviera más controlado.

Se dispone de dos conjuntos de datos, uno de los cuales se refiere al vino tinto y cuenta con 1.599 variedades diferentes, y el otro, al vino blanco, con 4.898 variedades. Todos los vinos se producen en una zona concreta de Portugal. Se recogen datos sobre 12 propiedades diferentes de los vinos, una de las cuales es la calidad, basada en datos sensoriales, y el resto son propiedades químicas de los vinos, como la densidad, la acidez, el contenido de alcohol, etc. Todas las propiedades químicas de los vinos son variables continuas. La calidad es una variable ordinal con una clasificación posible de 1 (peor) a 10 (mejor). Cada variedad de vino es catada por tres catadores independientes y la clasificación final asignada es la mediana de la clasificación dada por los catadores.

Se espera que un modelo predictivo desarrollado a partir de estos datos sirva de orientación a los viñedos en cuanto a la calidad y el precio que se espera obtener de sus productos sin depender en gran medida de la volatilidad de los catadores.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
from plotnine import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, make_scorer, roc_curve, auc
from scipy import stats
import seaborn as sns
from sklearn.datasets.samples_generator import make_blobs
from sklearn import preprocessing, metrics, model_selection
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.svm import SVC

In [ ]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [ ]:
data_r.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5


In [ ]:
data_w.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5,6


In [ ]:
data = data_w.assign(type = 'white')
data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
1248,7.2,0.18,0.31,1.10,0.045,20.0,73.0,0.99250,3.32,0.40,10.8,7,white
3024,7.0,0.13,0.37,12.85,0.042,36.0,105.0,0.99581,3.05,0.55,10.7,6,white


In [ ]:
data.shape

(6497, 13)

In [ ]:
tab_quality = data['quality'].value_counts().rename_axis('Calidad').reset_index(name='Cantidad')
tab_quality

,Calidad,Cantidad
0,6,2836
1,5,2138
2,7,1079
3,4,216
4,8,193
5,3,30
6,9,5


In [ ]:
tab_type = data['type'].value_counts().rename_axis('Tipo').reset_index(name='Cantidad')
tab_type

,Tipo,Cantidad
0,white,4898
1,red,1599


# Ejercicio 1

Mostrar la tabla de frecuencias de la calidad por tipo de vino.

In [ ]:
pd.crosstab(index=data['type'],columns=data['quality'], margins=True)

quality,3,4,5,6,7,8,9,All
type,,,,,,,,
red,10,53,681,638,199,18,0,1599
white,20,163,1457,2198,880,175,5,4898
All,30,216,2138,2836,1079,193,5,6497


# Regularización

# Ejercicio 2

* Entrenar una regresión lineal para predecir la calidad del vino (Continuo)

* Analice los coeficientes

* Evaluar el RMSE

In [ ]:
# 0: vino blanco y 1: vino tinto, para poder integrar la variable en el modelo
data['type'] = data.type.map({'white':0,'red':1})
data.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5,6,0


In [ ]:
X = data.drop(['quality'], axis =1 )
Y = data['quality']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=1234)

modelo = LinearRegression()
modelo.fit(X_train, Y_train)

print(modelo.intercept_)
coeficientes_modelo = pd.DataFrame(modelo.coef_, columns =["Coeficientes"])
coeficientes_modelo.index =  ["fixed acidity", "volatile acidity", "citric acid",
                              "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide",
                              "density", "pH", "sulphates", "alcohol", "type"]
print('Los coeficientes del modelo son:',coeficientes_modelo)
print(modelo.score(X,Y))

Y_predic = modelo.predict(X_test)
Y_pred = np.around(Y_predic)

print("El RMSE del modelo es: ",np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

100.60008348954668
Los coeficientes del modelo son:                       Coeficientes
fixed acidity             0.082870
volatile acidity         -1.481117
citric acid              -0.035535
residual sugar            0.060525
chlorides                -0.742306
free sulfur dioxide       0.004510
total sulfur dioxide     -0.001266
density                -100.095794
pH                        0.478051
sulphates                 0.722681
alcohol                   0.230528
type                      0.341976
0.2963647406991581
El RMSE del modelo es:  0.7922703501282297


*Los coeficientes Bi del modelo miden el cambio en la calidad del vino (variale Y) por cada cambio unitario en los atributos (variables Xi), entonces, se observa que el modelo en cuestion le da mayor peso a las variables "pH","sulphates","alcohol", y "type" con coeficientes 0.51, 0.69, 0.21, y 0.41 respectivamente, es decir, al aumentar una unidad en estas variables, la calidad del vino aumenta las mencionadas cantidades de Bi, entonces se puede decir que estas son las variables encargadas de incrementar la calidad del vino. Por otra parte, si los valores de los Bi -1.53, -0.13 y -109.7 que acompañan a las variables "volatile acidity", "citric acid", "chlorides" y "density" respectivamente, disminuyen, la calidad del vino tambien va a disminuir. Estas conclusiones parten teniendo en cuenta que para cada variable se asume como contantes las demas.*

In [ ]:
print("El RMSE del modelo es: ",np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

El RMSE del modelo es:  0.7883771061734742


*Se aprecio un valor muy alto de RMSE, esto indica que el modelo no tiene un buen ajuste y por lo tanto no genera una buena prediccion de la variable Y (quality).*

# Ejercicio 3

* Estimar una regresión de Ridge con alfa igual a 0,1 y 1.
* Compare y analice los coeficientes de Ridge con la regresión lineal
* Evaluar el RMSE

In [ ]:
X = data.drop(['quality'], axis =1 )
Y = data['quality']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=1234)

alfa = [0.1, 1]

intercepto_ridge = []
coeficientes_ridge = []
r2_ridge = []
RMSE_ridge = []

for it in alfa:
    modelo_ridge = Ridge(alpha=it, normalize=True)
    modelo_ridge.fit(X_train, Y_train)

    intercepto = (modelo_ridge.intercept_)
    intercepto_ridge.append(intercepto)

    coeficientes = (modelo_ridge.coef_)
    coeficientes_ridge.append(coeficientes)

    r2 = (modelo_ridge.score(X,Y))
    r2_ridge.append(r2)

    Y_predic = modelo_ridge.predict(X_test)
    Y_pred = modelo_ridge.predict(X_test)

print(pd.DataFrame(intercepto_ridge))
print('Los coeficientes de los modelos son:', np.array(coeficientes_ridge))
print( pd.DataFrame(r2_ridge))

RMSE = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred))
RMSE_ridge.append(RMSE)
print("El respectivo RMSE es:", pd.DataFrame(RMSE_ridge))

           0
0  39.185807
1  26.958817
Los coeficientes de los modelos son: [[ 2.41267556e-02 -1.23290300e+00  6.28214583e-02  2.84597398e-02
  -1.13568089e+00  4.11538161e-03 -1.37339205e-03 -3.71461231e+01
   2.13181461e-01  6.14602278e-01  2.58412490e-01  1.17268780e-01]
 [-1.64630760e-04 -5.74532464e-01  1.82441340e-01  5.57021917e-03
  -1.27223070e+00  1.46810287e-03 -5.74946130e-04 -2.29358069e+01
   8.10872512e-02  3.03020569e-01  1.39310622e-01 -1.85065543e-02]]
          0
0  0.289368
1  0.218769
El respectivo RMSE es:           0
0  0.767778


In [ ]:
# El mejor alfa
alpha_range = 10.**np.arange(-1, 1)
modelo_ridge_cv = RidgeCV(alphas=alpha_range, normalize=True, scoring='neg_mean_squared_error')
modelo_ridge_cv.fit(X_train, Y_train)
print('El mejor alfa el modelo es:',modelo_ridge_cv.alpha_)

El mejor alfa el modelo es: 0.1


In [ ]:
np.array(coeficientes_ridge)

array([[ 2.41267556e-02, -1.23290300e+00,  6.28214583e-02,
         2.84597398e-02, -1.13568089e+00,  4.11538161e-03,
        -1.37339205e-03, -3.71461231e+01,  2.13181461e-01,
         6.14602278e-01,  2.58412490e-01,  1.17268780e-01],
       [-1.64630760e-04, -5.74532464e-01,  1.82441340e-01,
         5.57021917e-03, -1.27223070e+00,  1.46810287e-03,
        -5.74946130e-04, -2.29358069e+01,  8.10872512e-02,
         3.03020569e-01,  1.39310622e-01, -1.85065543e-02]])

*Se aprecia que para algunas variables Xi los coeficientes Bi de la regresion lineal de ridge con alfa = 0.1 tienden a disminuir, con respecto a la regresión lineal calcula en el ejercicio 2, como es el caso de las variables "fixed acidity", "volatile acidity", "citric acid", "residual sugar", "density", "pH", "sulphates" y "type", por otra parte, los coeficientes -1.12 y 0.25 correspondientes a las variables "chlorides" y "alcohol" respectivamente, aumentan y por ultimo, tambien se aprecian Bi que no tienen un cambio significativo como lo son los de las variables "free sulfur dioxide" y "total sulfur dioxide", esto puede indicar que estos dos ultimos atributos no generan cambios en los modelos descritos anteriormente.*

# Ejercicio 4

* Estimar una regresión lasso con alfa igual a 0,01, 0,1 y 1.
* Comparar los coeficientes con la regresión lineal
* Evaluar el RMSE

In [ ]:
X = data.drop(['quality'], axis =1 )
Y = data['quality']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=1234)

alfa = [0.01, 0.1, 1]

intercepto_lasso = []
coeficientes_lasso = []
r2_lasso = []
RMSE_lasso = []

for it in alfa:
    modelo_lasso = Lasso(alpha=it, normalize=False)
    modelo_lasso.fit(X_train, Y_train)

    intercepto = (modelo_lasso.intercept_)
    intercepto_lasso.append(intercepto)

    coeficientes = (modelo_lasso.coef_)
    coeficientes_lasso.append(coeficientes)

    r2 = (modelo_lasso.score(X,Y))
    r2_lasso.append(r2)

    Y_predic = modelo_lasso.predict(X_test)
    Y_pred = modelo_lasso.predict(X_test)

print(pd.DataFrame(intercepto_lasso))
print('Los coeficientes de los modelos son:', np.array(coeficientes_lasso))
print(pd.DataFrame(r2_lasso))

RMSE = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred))
RMSE_lasso.append(RMSE)
print("El respectivo RMSE es:", pd.DataFrame(RMSE_lasso))

NameError: ignored

In [ ]:
# El mejor alfa
alpha_range = 10.**np.arange(-2, 1)
modelo_lasso_cv = LassoCV(alphas=alpha_range, normalize=True)
modelo_lasso_cv.fit(X_train, Y_train)
print('El mejor alfa para aplicar en el modelo es:',modelo_lasso_cv.alpha_)

In [ ]:
np.array(coeficientes_lasso)

*Como se conoce, la regresión de Lasso reduce los coeficientes para aquellas variables de entrada que no contribuyen mucho a la tarea de predicción, en este caso se observa que todos los coeficientes quedaron en 0 exceptuando el coeficiente -0.000351 que corresponde a la variable "total sulfur dioxide" que en comparación con la regresión lineal que se planteo inicialmente, era una variable de poca relevancia y con un coeficiente pequeño.*

# Ejercicio 5

* Estandarizar las características (excepto la calidad del vino)

* Crear una variable objetivo binaria para cada tipo de vino

* Analizar los coeficientes

* * Evalúe con F1, AUC-ROC y log-loss

In [ ]:
data_copy=data.copy()
data_trabajo=data_copy.drop(['quality','type'], axis=1)
data_trabajo.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.994,3.3,0.49,9.5


In [ ]:
quality=data['quality']
type=data['type']

In [ ]:
# Estandarizar datos
scaler = StandardScaler()
data_trabajo.astype(float)
scaler.fit(data_trabajo)
data_scaler=pd.DataFrame(scaler.transform(data_trabajo))
data_scaler.head(2)

,0,1,2,3,4,5,6,7,8,9,10
0,-0.166089,-0.423183,0.284686,3.206929,-0.314975,0.815565,0.959976,2.102214,-1.359049,-0.546178,-1.418558
1,-0.706073,-0.240949,0.147046,-0.807837,-0.200790,-0.931107,0.287618,-0.232332,0.506915,-0.277351,-0.831615


In [ ]:
data_trabajo=pd.concat([data_scaler,quality,type],axis=1)
data_trabajo.head(2)

,0,1,2,3,4,5,6,7,8,9,10,quality,type
0,-0.166089,-0.423183,0.284686,3.206929,-0.314975,0.815565,0.959976,2.102214,-1.359049,-0.546178,-1.418558,6,0
1,-0.706073,-0.240949,0.147046,-0.807837,-0.200790,-0.931107,0.287618,-0.232332,0.506915,-0.277351,-0.831615,6,0


In [ ]:
X = data_trabajo.drop(['type'], axis =1 )
Y = data_trabajo['type']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=1)
X_train = X_train.astype(float)
X_test = X_test.astype(float)

modelo_log = LogisticRegression(penalty='none',solver='lbfgs',multi_class='auto')
modelo_log.fit(X_train, Y_train)

print(modelo_log.intercept_)
coeficientes_modelo_log = pd.DataFrame(modelo_log.coef_)
coeficientes_modelo_log.columns=["fixed acidity", "volatile acidity", "citric acid",
                                "residual sugar", "chlorides", "free sulfur dioxide",
                                "total sulfur dioxide","density", "pH", "sulphates",
                                "alcohol", "quality"]
coeficientes_modelo_log.index =["Coeficientes"]
print('Los coeficientes del modelo son:',coeficientes_modelo_log.transpose())
print(modelo_log.score(X,Y))

Y_predic = modelo_log.predict(X_test)
Y_pred = np.around(Y_predic)

print("El RMSE del modelo es: ",np.sqrt(metrics.mean_squared_error(Y_test, Y_pred)))

[-7.97968791]
Los coeficientes del modelo son:                       Coeficientes
fixed acidity            -0.586792
volatile acidity          1.213229
citric acid              -0.387194
residual sugar           -5.318930
chlorides                 0.761295
free sulfur dioxide       0.810984
total sulfur dioxide     -2.729374
density                   5.592562
pH                       -0.321378
sulphates                 0.411828
alcohol                   2.098391
quality                   0.589264
0.993997229490534
El RMSE del modelo es:  0.1


*Los coeficientes que influyen positivamente en la respuesta corresponden a las variables "volatile acidity", "chlorides", "free sulfur dioxide", "density", "sulphates", "alcohol" y "quality".*

In [ ]:
# F1, AUC-ROC, LOG-LOSS
Y_pred_prob = modelo_log.predict_proba(X_test)
print('F1 score:', metrics.f1_score(Y_test, Y_pred))
print('AUC-ROC score:', metrics.roc_auc_score(Y_test, Y_pred, average=None))
print('Log-loss score:',metrics.log_loss(Y_test, Y_pred_prob))

F1 score: 0.9789303079416531
AUC-ROC score: 0.9824561403508772
Log-loss score: 0.0371537935181029


*La puntuación F1 esta muy cerca del mejor valor (1), esto indica que la presicion y la recuperación es alta. El AUC-ROC tambien es un valor alto, lo que indica que el predictor del modelo tiene la capacidad de distingir entre clases positivas y negativas. Por ultimo, el log-loss tiene un valor bajo cercano a 0 que indica mejores predicciones.*

# Ejercicio 6

* Estimar una regresión logística regularizada usando:
* C = 0.01, 0.1 & 1.0
* penalización = ['l1, 'l2']
* Compare los coeficientes y la puntuación f1.

Nota: Para los valores de C y Penalización, deben realizar todas las posibles combinaciones entre estas dos variables.

In [ ]:
# Penalizacion 'l1'
c = [0.01, 0.1, 1]

intercepto_log_l1 = []
coeficientes_log_l1 = []
r2_log_l1 = []
RMSE_log_l1 =[]
F1_log_l1 = []

for it in c:
  modelo_log_l1 = LogisticRegression(C=it, penalty='l1',solver='liblinear',multi_class='auto',random_state=1234)
  modelo_log_l1.fit(X_train, Y_train)

  intercepto = (modelo_log_l1.intercept_)
  intercepto_log_l1.append(intercepto)

  coeficientes = (modelo_log_l1.coef_)
  coeficientes_log_l1.append(coeficientes)

  r2 = (modelo_log_l1.score(X,Y))
  r2_log_l1.append(r2)

  Y_pred_prob = modelo_log_l1.predict_proba(X_test)
  RMSE = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred))
  RMSE_log_l1.append(RMSE)

  F1 = (metrics.f1_score(Y_test, Y_pred))
  F1_log_l1.append(F1)

print(pd.DataFrame(intercepto_log_l1))
print('Los coeficientes de los modelos son:',np.array(coeficientes_log_l1))
print(pd.DataFrame(r2_log_l1))
print(pd.DataFrame(RMSE_log_l1))
print('El respectivo F1 score es:', pd.DataFrame(F1_log_l1))

          0
0  0.000000
1  0.000000
2 -5.582226
Los coeficientes de los modelos son: [[[ 0.33265012  0.84538653  0.         -0.0420302   0.45419434
    0.         -1.73205739  0.41111489  0.35047584  0.60649738
    0.         -0.37206238]]

 [[ 0.          0.95232641 -0.12702655 -2.02615028  0.65235375
    0.26093091 -2.3308892   2.8557661   0.27507341  0.71279251
    0.97827985 -0.57030748]]

 [[-0.1841379   1.15867894 -0.34218087 -4.2504832   0.76403744
    0.69198176 -2.62626576  4.54750413  0.          0.47494256
    1.71159065  0.23413029]]]
          0
0  0.985378
1  0.992150
2  0.994305
     0
0  0.1
1  0.1
2  0.1
El respectivo F1 score es:          0
0  0.97893
1  0.97893
2  0.97893


In [ ]:
# Penalizacion 'l2'
c = [0.01, 0.1, 1]

intercepto_log_l2 = []
coeficientes_log_l2 = []
r2_log_l2 = []
RMSE_log_l2 =[]
F1_log_l2 = []

for it in c:
  modelo_log_l2 = LogisticRegression(C=it, penalty='l2',solver='liblinear',multi_class='auto',random_state=1234)
  modelo_log_l2.fit(X_train, Y_train)

  intercepto = (modelo_log_l2.intercept_)
  intercepto_log_l2.append(intercepto)

  coeficientes = (modelo_log_l2.coef_)
  coeficientes_log_l2.append(coeficientes)

  r2 = (modelo_log_l2.score(X,Y))
  r2_log_l2.append(r2)

  Y_pred_prob = modelo_log_l2.predict_proba(X_test)
  RMSE = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred))
  RMSE_log_l2.append(RMSE)

  F1 = (metrics.f1_score(Y_test, Y_pred))
  F1_log_l2.append(F1)


print(pd.DataFrame(intercepto_log_l2))
print('Los coeficientes de los modelos son:',np.array(coeficientes_log_l2))
print("")
print(pd.DataFrame(r2_log_l2))
print(pd.DataFrame(RMSE_log_l2))
print('El respectivo F1 score es:', pd.DataFrame(F1_log_l2))

          0
0 -0.175103
1 -0.731518
2 -3.027143
Los coeficientes de los modelos son: [[[ 0.59782433  0.75490584 -0.21287738 -0.55671999  0.61798162
   -0.22188947 -1.16476192  0.71398915  0.53883334  0.63910651
    0.16936178 -0.37238238]]

 [[ 0.73918326  1.03994202 -0.30790444 -1.35842685  0.84518557
    0.20946375 -2.12200253  1.66483577  0.68142031  0.83020656
    0.56260738 -0.43771165]]

 [[ 0.14167086  1.12329485 -0.33841614 -3.15113391  0.79769997
    0.6974427  -2.64695534  3.58135993  0.2429419   0.64162971
    1.37085605 -0.14733136]]]

          0
0  0.988610
1  0.992150
2  0.993843
     0
0  0.1
1  0.1
2  0.1
El respectivo F1 score es:          0
0  0.97893
1  0.97893
2  0.97893


In [ ]:
print('Los coeficientes de los modelos con l1 son:',np.array(coeficientes_log_l1))
print('Los coeficientes de los modelos con l2 son:',np.array(coeficientes_log_l2))

Los coeficientes de los modelos con l1 son: [[[ 0.33265012  0.84538653  0.         -0.0420302   0.45419434
    0.         -1.73205739  0.41111489  0.35047584  0.60649738
    0.         -0.37206238]]

 [[ 0.          0.95232641 -0.12702655 -2.02615028  0.65235375
    0.26093091 -2.3308892   2.8557661   0.27507341  0.71279251
    0.97827985 -0.57030748]]

 [[-0.1841379   1.15867894 -0.34218087 -4.2504832   0.76403744
    0.69198176 -2.62626576  4.54750413  0.          0.47494256
    1.71159065  0.23413029]]]
Los coeficientes de los modelos con l2 son: [[[ 0.59782433  0.75490584 -0.21287738 -0.55671999  0.61798162
   -0.22188947 -1.16476192  0.71398915  0.53883334  0.63910651
    0.16936178 -0.37238238]]

 [[ 0.73918326  1.03994202 -0.30790444 -1.35842685  0.84518557
    0.20946375 -2.12200253  1.66483577  0.68142031  0.83020656
    0.56260738 -0.43771165]]

 [[ 0.14167086  1.12329485 -0.33841614 -3.15113391  0.79769997
    0.6974427  -2.64695534  3.58135993  0.2429419   0.64162971
    1.

In [ ]:
print('F1 score con l1:',np.array(F1_log_l1))
print('F2 score con l2:',np.array(F1_log_l2))

F1 score con l1: [0.97893031 0.97893031 0.97893031]
F2 score con l2: [0.97893031 0.97893031 0.97893031]


# SVM

# Ejercicio 7

Teniendo en cuenta las mismas variables de la regresión logística:

* Crear un objetivo binario para cada tipo de vino
* Crear dos SVM lineales para los vinos blancos y tintos, respectivamente.


In [ ]:
# Variable objetivo para white
data['TypeW'] = data.type.map({'white':1,'red':0})
# Variable objetivo para red
data['TypeR'] = data.type.map({'white':0,'red':1})

In [ ]:
# Vino White
X = X
y = data['TypeW']
# split into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Construir un modelo de regresión lineal
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Vino White
from sklearn.svm import SVC
modSVCW = SVC(kernel='linear', C=1E6)
modSVCW.fit(X_train, y_train)

In [ ]:
# Vino Red
X = X
y = data['TypeR']
# split into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Construir un modelo de regresión lineal
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Vino Red
from sklearn.svm import SVC
modSVCW = SVC(kernel='linear', C=1E6)
modSVCW.fit(X_train, y_train)

# Ejercicio 8

* Probar las dos SVM utilizando los diferentes kernels ('poly', 'rbf', 'sigmoid')
* Evalúe con F1, AUC-ROC y log-loss


In [ ]:
##### VINO TINTO

## POLY
modSVC = SVC(kernel='poly', C=1E6)
modSVC.fit(X_train, y_train)

svc_clf = svm.SVC()
# Grilla de validación
parameters = {'gamma': np.linspace(0,1,20), 'C': np.linspace(10,1000,50)}
acc_scorer = make_scorer(roc_auc_score)

# Run and fit the grid search
grid_obj = GridSearchCV(svc_clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

grid_obj

svc_clf_best = grid_obj.best_estimator_
svc_clf_best

svc_clf_best.fit(X_train, y_train)

svc_pred = svc_clf_best.predict(X_test)
# F1
print('F1 score:', metrics.f1_score(y_test, svc_pred))
# AUC-ROC
print('ROC AUC score:', metrics.roc_auc_score(y_test, svc_pred, average=None))
# log-loss
print('Log Loss:', metrics.log_loss(y_test, svc_pred))

In [ ]:
## RBF
modSVC = SVC(kernel='rbf', C=1E6)
modSVC.fit(X_train, y_train)

svc_clf = svm.SVC()
# Grilla de validación
parameters = {'gamma': np.linspace(0,1,20), 'C': np.linspace(10,1000,50)}
acc_scorer = make_scorer(roc_auc_score)

# Run and fit the grid search
grid_obj = GridSearchCV(svc_clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

grid_obj
svc_clf_best = grid_obj.best_estimator_
svc_clf_best
svc_clf_best.fit(X_train, y_train)

svc_pred = svc_clf_best.predict(X_test)
# F1
print('F1 score:', metrics.f1_score(y_test, svc_pred))
# AUC-ROC
print('ROC AUC score:', metrics.roc_auc_score(y_test, svc_pred, average=None))
# log-loss
print('Log Loss:', metrics.log_loss(y_test, svc_pred))

In [ ]:
## SIGMOID
modSVC = SVC(kernel='sigmoid', C=1E6)
modSVC.fit(X_train, y_train)
svc_clf = svm.SVC()
# Grilla de validación
parameters = {'gamma': np.linspace(0,1,20), 'C': np.linspace(10,1000,50)}
acc_scorer = make_scorer(roc_auc_score)

# Run and fit the grid search
grid_obj = GridSearchCV(svc_clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

grid_obj
svc_clf_best = grid_obj.best_estimator_
svc_clf_best
svc_clf_best.fit(X_train, y_train)

svc_pred = svc_clf_best.predict(X_test)
# F1
print('F1 score:', metrics.f1_score(y_test, svc_pred))
# AUC-ROC
print('ROC AUC score:', metrics.roc_auc_score(y_test, svc_pred, average=None))
# log-loss
print('Log Loss:', metrics.log_loss(y_test, svc_pred))

In [ ]:
#### VINO BLANCO

## POLY
modSVC = SVC(kernel='poly', C=1E6)
modSVC.fit(X_train, y_train)
svc_clf = svm.SVC()
# Grilla de validación
parameters = {'gamma': np.linspace(0,1,20), 'C': np.linspace(10,1000,50)}
acc_scorer = make_scorer(roc_auc_score)
# Run and fit the grid search
grid_obj = GridSearchCV(svc_clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)
grid_obj

svc_clf_best = grid_obj.best_estimator_
svc_clf_best

svc_clf_best.fit(X_train, y_train)

svc_pred = svc_clf_best.predict(X_test)

# F1
print('F1 score:', metrics.f1_score(y_test, svc_pred))
# AUC-ROC
print('ROC AUC score:', metrics.roc_auc_score(y_test, svc_pred, average=None))
# log-loss
print('Log Loss:', metrics.log_loss(y_test, svc_pred))

In [ ]:
## RBF
modSVC = SVC(kernel='rbf', C=1E6)
modSVC.fit(X_train, y_train)

svc_clf = svm.SVC()
# Grilla de validación
parameters = {'gamma': np.linspace(0,1,20), 'C': np.linspace(10,1000,50)}
acc_scorer = make_scorer(roc_auc_score)

# Run and fit the grid search
grid_obj = GridSearchCV(svc_clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

grid_obj

svc_clf_best = grid_obj.best_estimator_
svc_clf_best
# F1
print('F1 score:', metrics.f1_score(y_test, svc_pred))
# AUC-ROC
print('ROC AUC score:', metrics.roc_auc_score(y_test, svc_pred, average=None))
# log-loss
print('Log Loss:', metrics.log_loss(y_test, svc_pred))

In [ ]:
## SIGMOID
modSVC = SVC(kernel='sigmoid', C=1E6)
modSVC.fit(X_train, y_train)

svc_clf = svm.SVC()
# Grilla de validación
parameters = {'gamma': np.linspace(0,1,20), 'C': np.linspace(10,1000,50)}
acc_scorer = make_scorer(roc_auc_score)

# Run and fit the grid search
grid_obj = GridSearchCV(svc_clf, parameters, scoring=acc_scorer)
grid_obj = grid_obj.fit(X_train, y_train)

grid_obj

svc_clf_best = grid_obj.best_estimator_
svc_clf_best

svc_clf_best.fit(X_train, y_train)

svc_pred = svc_clf_best.predict(X_test)
# F1
print('F1 score:', metrics.f1_score(y_test, svc_pred))
# AUC-ROC
print('ROC AUC score:', metrics.roc_auc_score(y_test, svc_pred, average=None))
# log-loss
print('Log Loss:', metrics.log_loss(y_test, svc_pred))

*El mejor SVM es el que utiliza kernel polinomico*

# Ejercicio 9
* Utilizando el mejor SVM encontrar los parámetros que da el mejor rendimiento teniendo en cuenta los siguientes hiperparámetros
'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]
* Evalúe cada modelo con F1, AUC-ROC y log-loss

Nota: Para los valores de C y gamma, deben realizar todas las posibles combinaciones entre estas dos variables.

In [ ]:
c = [0.1, 1, 10, 100, 1000]
Gama = [0.01, 0.001, 0.0001]

F1 = []
ROC = []
LOG = []

VC = []
GAMA = []


for i, j in itertools.product(c, Gama):

    mod = svm.SVC(C = i, gamma = j, kernel = "poly")

    mod.fit(X_train, y_train)

    y_pre = mod.predict(X_test)

    VF1 =  metrics.f1_score(y_test, y_pre)
    VROC= metrics.roc_auc_score(y_test2, y_pre, average=None)
    VLOG = metrics.log_loss(y_test, y_pre)


    F1.append(VF1)
    ROCappend(VROC)
    LOG.append(VLOG)


    VC.append(mod.C)
    GAMA.append(mod.gamma)

In [ ]:
combinaciones_parametros = pd.concat([pd.DataFrame(VC), pd.DataFrame(GAMA), pd.DataFrame(F1), pd.DataFrame(ROC), pd.DataFrame(LOG)], axis=1)
combinaciones_parametros.columns = ["Valor C", "Gamma", "F1", "AUC ROC", "LOG LOSS"]
print(combinaciones_parametros)

In [ ]:
# Mejor combinacion de hiperparametros
mejor_combinaciones_parametros = GridSearchCV(estimator = mod, param_grid = dict(C = C, gamma = Gama))
mejor_combinaciones_parametros.fit(X_train, y_train)
print(mejor_combinaciones_parametros.best_params_)

# Ejercicio 10

Compare los resultados con todos los anteriores modelos y eliga ¿Cuál algoritmo con cuáles hiperparámetros es el que tiene mejor rendimiento según la puntuación F1, AUC-ROC y log-loss?